In [14]:
import pandas as pd
from datetime import datetime
from unidecode import unidecode

def trim_overspace(text):
	parts = filter(lambda x: len(x) > 0,text.split(" "))
	return " ".join(parts)

def normalize_text(text):
    #.replace("'"," ") alguns municipios tem ' e as macros maravilhosas do excel vão falhar
	x = str(text).replace(".","").replace("\n","").replace(",","").upper()
	x = trim_overspace(x)
	x = unidecode(x)

	if x == 'nan' or len(x) == 0 or x == '0' or x == 'nao_informado':
		return ''
	else:
		return x
    
def normalize_labels(text):
	x = str(text).replace("'"," ").replace(".","").replace("\n","").replace(",","").lower()
	x = trim_overspace(x).replace(" ","_")
	x = unidecode(x)

	if x == 'nan' or len(x) == 0 or x == '0' or x == 'nao_informado':
		return ''
	else:
		return x
    
def normalize_number(num,cast=int,error='fill',fill='0'):
	try:
		return cast(num)
	except ValueError:
		if error == 'raise':
			raise Exception(ValueError)
		elif error == 'fill':
			return normalize_number(fill,cast,'raise')

In [2]:
casos_raw = pd.read_excel('novos_casos_raw.xlsx')

In [3]:
casos_raw.columns = [ normalize_labels(x) for x in casos_raw.columns]
casos_raw['nome_paciente'] = casos_raw['nome_paciente'].apply(normalize_text)
casos_raw['idade'] = casos_raw['idade'].apply(int)
casos_raw['municipio_residencia'] = casos_raw['municipio_residencia'].apply(normalize_text)
casos_raw['municipio_atendimento'] = casos_raw['municipio_atendimento'].apply(normalize_text)

In [4]:
casos_raw.dtypes

data_notificacao          datetime64[ns]
id_ficha                           int64
nome_paciente                     object
sexo                              object
idade                              int64
municipio_residencia              object
municipio_atendimento             object
rs                                object
exame                             object
data_diagnostico          datetime64[ns]
data_primeiro_sintomas    datetime64[ns]
dtype: object

In [5]:
casos_raw['hash'] = casos_raw.apply(lambda row: (row['nome_paciente'] + str(row['idade']) + row['municipio_residencia']).replace(" ",""), axis=1)
casos_raw['hash_less'] = casos_raw.apply(lambda row: (row['nome_paciente'] + str(row['idade']-1) + row['municipio_residencia']).replace(" ",""), axis=1)
casos_raw['hash_more'] = casos_raw.apply(lambda row: (row['nome_paciente'] + str(row['idade']+1) + row['municipio_residencia']).replace(" ",""), axis=1)

In [6]:
casos_raw = casos_raw.sort_values(by='nome_paciente')

In [7]:
print(f"casos novos: {casos_raw.shape[0]}")
print(f"casos novos duplicados: {casos_raw.loc[casos_raw.duplicated(subset='hash')].shape[0]}")
casos_raw = casos_raw.drop_duplicates(subset='hash')
print(f"casos novos sem duplicados: {casos_raw.shape[0]}")

casos novos: 5122
casos novos duplicados: 2
casos novos sem duplicados: 5120


In [8]:
casos_confirmados = pd.read_excel('Casos confirmados PR.xlsx', 'Casos confirmados', usecols='D:M')
casos_confirmados.columns = [ normalize_labels(x) for x in casos_confirmados.columns]

In [9]:
casos_confirmados['nome'] = casos_confirmados['nome'].apply(normalize_text)
casos_confirmados['idade'] = casos_confirmados['idade'].apply(normalize_number)
casos_confirmados['mun_resid'] = casos_confirmados['mun_resid'].apply(normalize_text)
casos_confirmados['mun_atend'] = casos_confirmados['mun_atend'].apply(normalize_text)
casos_confirmados['hash'] = casos_confirmados.apply(lambda row: (row['nome'] + str(row['idade']) + row['mun_resid']).replace(" ",""), axis=1)
casos_confirmados.dtypes

nome                   object
sexo                   object
idade                   int64
mun_resid              object
mun_atend              object
rs                    float64
laboratorio            object
dt_diag                object
comunicacao    datetime64[ns]
is                     object
hash                   object
dtype: object

In [10]:
index_casos_duplicados = casos_raw.loc[casos_raw['hash'].isin(casos_confirmados['hash'])].index.to_list()
print(f"casos já em casos com a mesma idade: {len(index_casos_duplicados)}")
index_casos_duplicados_idade_less = casos_raw.loc[casos_raw['hash_less'].isin(casos_confirmados['hash'])].index.to_list()
print(f"casos já em casos com a idade - 1: {len(index_casos_duplicados_idade_less)}")
index_casos_duplicados_idade_more = casos_raw.loc[casos_raw['hash_more'].isin(casos_confirmados['hash'])].index.to_list()
print(f"casos já em casos com a idade + 1: {len(index_casos_duplicados_idade_more)}")
print(f"dentre os quais {len(set(index_casos_duplicados_idade_more).intersection(index_casos_duplicados)) + len(set(index_casos_duplicados_idade_less).intersection(index_casos_duplicados))} são casos em comum, o que leva a crer que estão duplicados na planilha já com idade a mais ou idade a menos")
index_duplicados = list(set(index_casos_duplicados + index_casos_duplicados_idade_less + index_casos_duplicados_idade_more))
print(f"sendo assim, {len(index_duplicados)} casos que já se encontram na planilha serão removidos")

casos já em casos com a mesma idade: 1966
casos já em casos com a idade - 1: 114
casos já em casos com a idade + 1: 1
dentre os quais 15 são casos em comum, o que leva a crer que estão duplicados na planilha já com idade a mais ou idade a menos
sendo assim, 2066 casos que já se encontram na planilha serão removidos


In [11]:
print(f"então, de {len(casos_raw)} casos baixados hoje  {len(casos_raw)-len(index_duplicados)} serão adicionados")
casos_raw = casos_raw.drop(index=index_duplicados)
casos_raw.shape

então, de 5120 casos baixados hoje  3054 serão adicionados


(3054, 14)

In [30]:
casos_raw['data_com'] = [ "" ] * len(casos_raw)
casos_raw['rs'] = [ "" ] * len(casos_raw)
casos_raw.columns

Index(['data_notificacao', 'id_ficha', 'nome_paciente', 'sexo', 'idade',
       'municipio_residencia', 'municipio_atendimento', 'rs', 'exame',
       'data_diagnostico', 'data_primeiro_sintomas', 'hash', 'hash_less',
       'hash_more', 'data_com'],
      dtype='object')

In [31]:
novos_casos = casos_raw[['nome_paciente','sexo','idade','municipio_residencia', 'municipio_atendimento', 'rs', 'exame','data_diagnostico','data_com','data_primeiro_sintomas']]
novos_casos.to_excel('novos_casos.xlsx', index=False)